In [14]:
from scikeras.wrappers import KerasClassifier

In [15]:
import pandas  as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import  Dense

In [16]:
data = pd.read_csv('Churn_Modelling.csv')

In [17]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [18]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [19]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])



In [20]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
## combine one hot encoded column with orginal data 
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1) 
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
## combine one hot encoded column with orginal data 
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)  

In [21]:
## divide the  data set into dependent and independent data set
X=data.drop('Exited',axis=1)
y = data['Exited']

## split the data in training and testing data
X_train ,X_test,y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
## scale these feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('label_encoder_gender.pk1','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pk1','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pk1','wb') as file:
    pickle.dump(scaler,file)    

In [22]:
## define as function to create model and try different parameters (Keras Classifier)

def create_model(neurons =32 ,layers =1):
    model =Sequential()
    model.add(Dense(neurons,activation='relu',input_shape = (X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
    


In [23]:
## create a keras classifier
model = KerasClassifier(neurons =32 ,layers =1,build_fn=create_model,epochs=50,batch_size=10,verbose=0)


In [24]:
param_grid = {
    'neurons' : [16,32,64,128],
    'layers' : [1,2],
    'epochs' : [50,100]
}

In [25]:
## perform grid search
from sklearn.model_selection import GridSearchCV
 
grid = GridSearchCV(estimator= model ,param_grid =param_grid,n_jobs = -1 ,cv=3)
grid_result = grid.fit(X_train,y_train)

## print the best parameters
print("Best : %f using %s" % (grid_result.best_score_ , grid_result.best_params_))

d:\Learn Machine learning\Prediction_Based_on_customer_data\prediction_env\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)






Best : 0.858749 using {'epochs': 50, 'layers': 1, 'neurons': 128}


In [26]:
## build ANN models
model  = Sequential([
    ### Input(shape=(X_train.shape[1],)),  # Define input shape using Input()  chatgpt

    Dense(64,activation='relu',input_shape= (X_train.shape[1],)),  ## hl1
    Dense(1,activation='sigmoid')   ## output  function
]

)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                832       
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 897 (3.50 KB)
Trainable params: 897 (3.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
### inorder to do froward and backward propogation compile the model
import tensorflow
#from tensorflow.keras.optimizers import Adam
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
from tensorflow.keras.losses import BinaryCrossentropy
loss = BinaryCrossentropy()

In [33]:
i=1

In [34]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "log/fit" + "training" + "1"
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [35]:
import numpy as np
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
X_train_array = np.array(X_train)
y_train_array = np.array(y_train)
X_test_array = np.array(X_test)
y_test_array = np.array(y_test)


In [36]:
## training model
history =model.fit(
    X_train,y_train,validation_data=(X_test,y_test),
    epochs=50,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
250/250 [==============================] - 2s 4ms/step - loss: 0.3925 - accuracy: 0.8353 - val_loss: 0.3533 - val_accuracy: 0.8530
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3598 - accuracy: 0.8529 - val_loss: 0.3664 - val_accuracy: 0.8475
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3533 - accuracy: 0.8580 - val_loss: 0.3451 - val_accuracy: 0.8575
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3498 - accuracy: 0.8566 - val_loss: 0.3442 - val_accuracy: 0.8635
Epoch 5/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3455 - accuracy: 0.8547 - val_loss: 0.3437 - val_accuracy: 0.8530
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3429 - accuracy: 0.8564 - val_loss: 0.3469 - val_accuracy: 0.8540
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8579 - val_loss: 0.3445 - val_accuracy: 0.8545
Epoch 